In [ ]:
from dotenv import load_dotenv
import os 
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv('env', override=True)
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
END_POINT=os.getenv('END_POINT')
MODEL_NAME=os.getenv('MODEL_NAME')
# print(AZURE_OPENAI_API_KEY[:10])
print(MODEL_NAME)

AZURE_OPENAI_EMB_API_KEY = os.getenv('AZURE_OPENAI_EMB_API_KEY')
EMB_END_POINT=os.getenv('EMB_END_POINT')
EMB_MODEL_NAME=os.getenv('EMB_MODEL_NAME')

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_TRACING_V2'] = 'true' #true, false
os.environ['LANGCHAIN_PROJECT'] = 'LANG'

if os.getenv('LANGCHAIN_TRACING_V2') == "true":
    # print('랭스미스로 추적 중입니다 :', os.getenv('LANGSMITH_API_KEY')[:10])

43b13g4OZS
gpt-4.1-mini
랭스미스로 추적 중입니다 : lsv2_pt_24


## 1. 간단한 IT 용어 사전 만들기

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,          
    api_version="2024-12-01-preview",
    temperature=0.2,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 it 관련 용어를 설명해주는 도우미다. 답변은 한국어로. 간결하고 정확하게. 형식은 '용어: 설명' 형태로 작성하라."),
    ("human", "다음 IT 용어를 설명하라: {term}")
])

chain = {"term" : RunnablePassthrough()} | prompt | llm | StrOutputParser()

chain.invoke('대역폭')

'대역폭: 네트워크나 통신 회선이 일정 시간 내에 전송할 수 있는 데이터의 최대량을 의미하며, 보통 초당 비트 수(bps)로 측정된다.'

## 2. 고객 요청사항 요약 봇

In [4]:
from typing import List, Literal
from pydantic import BaseModel, Field
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 1) 스키마 정의
class Summary(BaseModel):
    등록시간: str = Field(..., description="등록 시간") #...은 누락이 되면 안되는 필수 필드를 의미함
    핵심문제: str = Field(..., description="고객이 겪는 핵심 문제")
    요청사항: str = Field(..., description="고객이 원하는 조치/해결")
    분류: Literal["문제해결","기능요청","결제/계정","성능","문의/가이드","기타"] #Literal은 문자열 리터럴 타입을 의미함
    태그: List[str] = Field(..., min_length=3, max_length=5, description="소문자, 언더바(_) 사용")
    키워드: List[str] = Field(..., min_length=3, max_length=5, description="핵심 명사/용어")

# 2) LLM
llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,          
    api_version="2024-12-01-preview",
    temperature=0.2,
)

# 3) 구조화 출력용 Runnable, output_parser는 필요없음
structured_llm = llm.with_structured_output(Summary)

# 4) 프롬프트
SYSTEM = (
    "너는 고객 요청 요약 전문가다. 반드시 한국어로 답하고, 아래 기준을 지켜라.\n"
    "- 등록 시간 : 현재 시간을 KST 기준으로 yearY/monthM/dayD/hourH/minutem 형식으로 작성하라.\n"
    "- 핵심문제, 요청사항: 간결하고 명확하게. 명사형 어미로 끝내라 ex) 발생, 문제, 문의, 요청, 해결\n"
    "- 분류: 6개 중 하나 이상으로 엄격히 선택하라\n"
    "- 태그: 1~3개, 소문자, 언더바(_) 사용\n"
    "- 키워드: 3~5개, 핵심 명사/용어, 가능한 요청 내부에 존재하는 단어를 사용\n"
    "- 정보 부족 시 '미상' 사용, 개인정보 미포함"
)
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM),
    ("human", "다음 고객 요청을 요약해줘:\n{request}, 등록시간: {time}")
])

# 5) 체인
chain = prompt | structured_llm

In [5]:
import json
from datetime import datetime

ticket_text = """어제부터 앱 로그인에 계속 실패합니다. 비밀번호를 새로 바꿨는데도 안되네요. 고객 센터 게시판을 봐도 잘 모르겠고 빨리 해결됐으면 합니다. 사용 중인 기기는 아이폰입니다."""
time = datetime.now().strftime("%Y-%m-%d-%H-%M")
output = chain.invoke({"request": ticket_text, "time": time})

print(json.dumps(output.model_dump(), ensure_ascii=False, indent=2))

{
  "등록시간": "2025Y/10M/16D/10H/29m",
  "핵심문제": "앱 로그인 실패 문제",
  "요청사항": "로그인 문제 해결 요청",
  "분류": "문제해결",
  "태그": [
    "로그인",
    "아이폰",
    "비밀번호"
  ],
  "키워드": [
    "앱",
    "로그인",
    "비밀번호",
    "아이폰",
    "고객센터"
  ]
}


## 3. 예의바른 번역기

In [6]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,          
    api_version="2024-12-01-preview",
    temperature=0.2,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 다양한 인간 언어를 자유자재로 번역하는 번역기야."
    " {input_language}로 된 문장을 {output_language}로 번역해줘. 최대한 자연스럽고 매끄럽게 번역해줘. "
    "회사에서 사용하는 목적이기 때문에 비즈니스 상황에 맞게 번역해줘."
    "이모티콘 등 비즈니스 상황과 맞지 않는 불필요한 요소가 있다면 제거해줘"
    "줄 바꿈 및 문단 구분이 필요하면 적절히 처리하라"
    ),
    ("human", "다음 용어를 번역해줘: {text}")
])

prompt = prompt.partial(input_language="한국어",
                        output_language="영어",)

chain = {"text" : RunnablePassthrough()} | prompt | llm | StrOutputParser()

text = "안녕? 내일 만나기로 했었는데 내가 일이 생겨서 못 갈 것 같아. 너무 미안해. 아직 정확하게 언제 시간이 될지 모르겠어. 다음 주 중에는 꼭 보자!. 시간 정해지면 바로 연락할게~ 안뇽~."

for chunk in chain.stream(text):
    print(chunk, end='', flush=True)

Hello, we were supposed to meet tomorrow, but something came up at work, so I don’t think I’ll be able to make it. I’m very sorry. I’m not sure exactly when I’ll be available yet. Let’s definitely meet sometime next week. I’ll contact you as soon as I have a confirmed time.

In [7]:
query = {
    "text": "today was fun 🤗 i’ll send a short recap soon—just say ‘got it’ when it lands ✉️ and tweak anything wrong & tell me! see ya~ 😎",
    "input_language": "영어",
    "output_language": "한국어"
}

for chunk in chain.stream(query):
    print(chunk, end='', flush=True)

오늘 즐거웠습니다. 곧 간단한 요약을 보내드리겠습니다. 도착하면 ‘확인했습니다’라고만 답해주시고, 잘못된 부분이 있으면 수정해 알려주세요. 곧 뵙겠습니다.

## 4. 고객 상담 자동 QA 만들기 (Few-shot)

Few-shot은 LLM이 답할 때 참고할 예시를 프롬프트에 포함해 형식이나 사고 절차, 출력 구조를 따르게 하는 기법입니다.

In [3]:
from collections import Counter

from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

# 0) 카테고리 & 연락처 디렉터리
CATEGORIES = ["버그/장애","기능요청","결제/계정","성능","문의/가이드","기타"]
CONTACTS = {
    "버그/장애":    {"team":"고객지원(장애)", "email":"support@acme.example", "phone":"02-123-0551", "hours":"상시 대응"},
    "기능요청":     {"team":"제품기획",       "email":"pm@acme.example",      "phone":"02-481-0011", "hours":"09:00-18:00 KST"},
    "결제/계정":    {"team":"청구/계정",      "email":"billing@acme.example", "phone":"02-234-1003", "hours":"09:00-18:00 KST"},
    "성능":        {"team":"SRE/성능",       "email":"sre@acme.example",     "phone":"02-834-0054", "hours":"09:00-18:00 KST"},
    "문의/가이드":  {"team":"고객지원(가이드)","email":"help@acme.example",    "phone":"02-952-8005", "hours":"09:00-18:00 KST"},
    "기타":        {"team":"일반문의/영업",   "email":"contact@acme.example", "phone":"02-114-0306", "hours":"09:00-18:00 KST"},
}

FAQS = [
    # ── 버그/장애 ─────────────────────────────────────────────
    {"question":"기가인터넷이 자주 끊깁니다. 모뎀 불이 빨간색이에요.",
     "answer":"전원/광단자/랜선 연결을 순서대로 점검하고, 10초 전원 재부팅을 진행해 주세요. 지속 시 장애 접수해주세요.",
     "category":"버그/장애"},
    {"question":"IPTV 화면이 까맣고 소리만 나와요.",
     "answer":"셋톱박스와 TV의 HDMI 케이블을 재결합하고, 셋톱/TV 전원을 순서대로 재부팅해 보세요. 해상도(1080p/2160p)도 확인해 주세요.",
     "category":"버그/장애"},
    {"question":"와이파이에 연결되지만 인터넷이 안 됩니다.",
     "answer":"공유기 WAN(인터넷) 포트 연결과 외부 회선 상태를 확인해 주세요. 모뎀→공유기→단말 순서로 재부팅을 권장합니다.",
     "category":"버그/장애"},
    {"question":"통화가 자주 끊기고 음질이 안 좋아요.",
     "answer":"VoLTE/5G 우선 설정을 확인하고, 특정 지역 이슈인지 상태 공지를 점검해 주세요. 심하면 기지국 점검을 위해 장애 접수 바랍니다.",
     "category":"버그/장애"},

    # ── 기능요청 ─────────────────────────────────────────────
    {"question":"가족 결합 회선을 더 추가하고 싶어요.",
     "answer":"가족관계 증빙 후 결합 회선 추가가 가능합니다. 명의자 인증과 서류 제출 경로를 안내해 드릴 수 있어요.",
     "category":"기능요청"},
    {"question":"IPTV 자막 크기/색상을 바꾸는 기능이 있으면 좋겠어요.",
     "answer":"접근성 향상을 위한 UI 개선 요청으로 접수 가능해요. 선호 크기/색상/콘텐츠 유형을 함께 알려주세요.",
     "category":"기능요청"},
    {"question":"eSIM 보조 회선을 추가 지원해 주세요.",
     "answer":"단말 호환 여부와 다회선 정책에 따라 검토됩니다. 사용 목적과 예상 데이터 사용량을 공유해 주세요.",
     "category":"기능요청"},
    {"question":"멤버십 포인트를 자동으로 청구서에 차감하고 싶어요.",
     "answer":"포인트 자동 적용 옵션은 정책에 따라 순차 제공됩니다. 현재는 청구 확정 전 수동 적용을 안내드립니다.",
     "category":"기능요청"},

    # ── 결제/계정 ────────────────────────────────────────────
    {"question":"자동이체 계좌를 변경하고 싶습니다.",
     "answer":"본인 인증 후 앱/웹 ‘납부방법 변경’에서 계좌를 수정할 수 있어요. 변경 후 다음 청구분부터 반영됩니다.",
     "category":"결제/계정"},
    {"question":"청구서 이메일 주소를 바꾸고 싶어요.",
     "answer":"마이페이지 > 청구서 수신 설정에서 이메일을 수정하세요. 수정 즉시 다음 고지부터 적용됩니다.",
     "category":"결제/계정"},
    {"question":"연체가 있는데 분할 납부 가능한가요?",
     "answer":"연체 기간/금액에 따라 분할 납부가 가능합니다. 본인 확인 후 납부 계획 수립을 도와드려요.",
     "category":"결제/계정"},
    {"question":"명의 변경(상속/양도)을 하고 싶습니다.",
     "answer":"필요 서류(신분증, 관계 증빙 등) 지참 후 명의 변경이 가능합니다. 상황별 서류 목록과 접수 채널을 안내해 드려요.",
     "category":"결제/계정"},

    # ── 성능 ────────────────────────────────────────────────
    {"question":"5G가 자꾸 LTE로 전환돼요.",
     "answer":"건물 내부/지하 등 전파 환경 영향일 수 있어요. 지역/시간대 로그를 제공해 주시면 커버리지 점검을 요청합니다.",
     "category":"성능"},
    {"question":"지하철에서 다운로드 속도가 너무 느립니다.",
     "answer":"혼잡 시간대 셀 부하 가능성이 있어요. 노선/시간 정보를 주시면 용량 증설 대상 검토에 반영됩니다.",
     "category":"성능"},
    {"question":"게임할 때 핑이 튀어요.",
     "answer":"DNS 고정, 백그라운드 앱 정리, 5GHz Wi-Fi 사용을 권장합니다. 서버 지역/시간대 로그를 주시면 추가 분석 가능합니다.",
     "category":"성능"},
    {"question":"해외 로밍 데이터 속도가 일정치 않습니다.",
     "answer":"제휴망/지역에 따라 속도가 상이합니다. 수동 사업자 선택/네트워크 모드 변경으로 개선될 수 있어요.",
     "category":"성능"},

    # ── 문의/가이드 ─────────────────────────────────────────
    {"question":"이사로 인터넷/티비 이전 설치를 신청하고 싶어요.",
     "answer":"이전 주소/희망 일자를 알려주시면 설치 가능 여부와 예약 일정을 안내드려요. 장비 반납/재사용도 함께 점검합니다.",
     "category":"문의/가이드"},
    {"question":"AS 방문 예약을 변경하고 싶습니다.",
     "answer":"예약 변경은 방문 전일까지 가능해요. 가능한 날짜/시간대를 알려주시면 재예약을 도와드려요.",
     "category":"문의/가이드"},
    {"question":"국제로밍 설정 방법(iOS/Android)을 알려주세요.",
     "answer":"설정 > 셀룰러/모바일 데이터에서 로밍을 활성화하고, 데이터 옵션을 확인하세요. 요금제/일 상한도 함께 안내드립니다.",
     "category":"문의/가이드"},
    {"question":"IPTV 리모컨이 TV와 연동이 안 됩니다.",
     "answer":"건전지 교체 후 페어링 모드로 진입하여 TV 브랜드 코드를 입력해 주세요. 모델별 코드표 링크를 제공해 드립니다.",
     "category":"문의/가이드"},

    # ── 기타 ────────────────────────────────────────────────
    {"question":"약정 만료일과 위약금이 궁금합니다.",
     "answer":"약정 종료일/잔여 약정/위약금은 본인 인증 후 조회 가능합니다. 요금제 변경 시 영향도 함께 안내해 드려요.",
     "category":"기타"},
    {"question":"엔터프라이즈 회선 계약서(SLA 포함)가 필요합니다.",
     "answer":"B2B 전용 계약/SLA 문서 제공이 가능합니다. 회사명/사용 규모/요구 SLA를 알려주시면 담당 부서로 연결합니다.",
     "category":"기타"},
    {"question":"개인정보 열람/삭제를 요청하고 싶어요.",
     "answer":"본인 확인 후 개인정보 열람/삭제/정지 요청이 가능합니다. 처리 범위와 소요 기간을 안내드립니다.",
     "category":"기타"},
    {"question":"스팸/스미싱 문자를 신고하고 싶습니다.",
     "answer":"국번 없이 118 또는 전용 앱에서 신고 가능해요. 발신번호/수신 시각/내용 캡처를 함께 제출해 주세요.",
     "category":"기타"},

]

# Example Selector (질문 유사도 기반으로 k개 선택)
emb = AzureOpenAIEmbeddings(
    model=EMB_MODEL_NAME,                      
    api_key=AZURE_OPENAI_EMB_API_KEY,
    azure_endpoint=EMB_END_POINT,
    api_version="2024-08-01-preview"
)

selector = SemanticSimilarityExampleSelector.from_examples(
    examples=FAQS,
    embeddings=emb,
    vectorstore_cls=FAISS,
    k=3,
    input_keys=["question"],  
)

# 예시를 대화형 포맷으로 렌더링함
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Q: {question}"),
    ("ai",    "A: {answer}\n분류: {category}")
])

fewshot = FewShotChatMessagePromptTemplate(
    example_selector=selector,
    example_prompt=example_prompt,
)


In [1]:
# 검색된 예시를 사람과 ai가 대화했던 과거 채팅 기록처럼 렌더링함
fewshot.invoke({"question": "연체가 됐어요. 어떡하죠?"})

NameError: name 'fewshot' is not defined

In [35]:
run('연체가 됐어요. 어떡하죠?')

1) 연체 금액과 기간에 따라 분할 납부가 가능하며, 본인 확인 후 납부 계획을 안내해 드립니다.  
2) 빠른 해결을 위해 고객센터에 문의하거나 앱/웹에서 납부 상태를 확인해 주세요.  

To-do:  
- 고객센터 연락 또는 앱/웹 로그인 후 납부 내역 확인  
- 분할 납부 신청 및 납부 계획 상담 요청

---
### 담당자 연락처
- 분류: 결제/계정
- 팀: 청구/계정
- 이메일: billing@acme.example
- 전화: 02-234-1003
- 운영시간: 09:00-18:00 KST

### 참고한 FAQ
- 연체가 있는데 분할 납부 가능한가요? (결제/계정)
- 자동이체 계좌를 변경하고 싶습니다. (결제/계정)
- 명의 변경(상속/양도)을 하고 싶습니다. (결제/계정)


In [36]:
run("5G인데 너무 느려요")

1) 5G 속도 저하는 기지국 혼잡, 전파 간섭 등 환경 요인 때문일 수 있습니다.  
2) 사용 위치와 시간 정보를 알려주시면 속도 개선 검토에 도움이 됩니다.  
3) 필요 시 네트워크 상태 점검을 위해 장애 접수를 권장합니다.  

To-do:  
- 사용 위치 및 시간대 정보 수집  
- 네트워크 상태 확인 및 장애 접수 안내

---
### 담당자 연락처
- 분류: 성능
- 팀: SRE/성능
- 이메일: sre@acme.example
- 전화: 02-834-0054
- 운영시간: 09:00-18:00 KST

### 참고한 FAQ
- 5G가 자꾸 LTE로 전환돼요. (성능)
- 지하철에서 다운로드 속도가 너무 느립니다. (성능)
- 통화가 자주 끊기고 음질이 안 좋아요. (버그/장애)


## IT 용어 사전

최신ICT시사용어2025.pdf를 업로드하세요

### 1. vector storage와 retriever 만들기

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

def build_vectorDB(pdf_path):
    """PDF를 읽어 청크로 나누고, 임베딩하여 검색기를 만든다."""
    # 데이터 로드
    loader = PyMuPDFLoader(pdf_path)

    # 텍스트 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800, chunk_overlap=120,
        separators=["\n\n", "\n", " ", ""]
    )
    
    chunks = loader.load_and_split(text_splitter=splitter)

    # 임베딩 & 인덱싱
    emb = AzureOpenAIEmbeddings(
        model=EMB_MODEL_NAME,                      
        api_key=AZURE_OPENAI_EMB_API_KEY,
        azure_endpoint=EMB_END_POINT,
        api_version="2024-08-01-preview"
    )
    vectorDB = FAISS.from_documents(chunks, emb)

    # 검색기 반환
    return vectorDB

PDF_PATH = "./pdf/최신ICT시사용어2025.pdf"   # 해당 경로에 파일이 있어야 합니다.
K = 6                                     # 검색 상위 청크 개수

vectorDB = build_vectorDB(PDF_PATH)
retriever = vectorDB.as_retriever(search_kwargs={"k": K})

### 2. RAG 랭체인 만들기

In [43]:
SYSTEM = """너는 '용어집 RAG 도우미'다.
반드시 제공된 컨텍스트 안에서만 한국어로 답하라.
컨텍스트에 없으면 '용어사전에서 찾을 수 없습니다.'이라고 답하라.

출력 형식:
<정의>: 2~4문장으로 간단하게 작성
<참고>: 사용한 출처의 페이지번호 
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM),
    ("human",
     "질문(용어): {question}\n\n"
     "아래 컨텍스트만 근거로 답하라.\n"
     "=== 컨텍스트 시작 ===\n{context}\n=== 컨텍스트 끝 ===\n")
])

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,          
    api_version="2024-12-01-preview",
    temperature=0.2,
)

chain = prompt | llm | StrOutputParser()


def format_with_citations(docs):
    """메타 데이터를 출력 형식에 맞게 문자열로 변환"""
    lines = []
    for _, d in enumerate(docs):
        page = int(d.metadata.get("page", 0)) + 1  # 페이지는 1부터 표기
        lines.append(f"<page> p.{page} </page> <contents>{d.page_content}</contents>")  # 컨택스트를 구조화 하면 성능이 좋아질 수 있음
    return "\n\n".join(lines)


In [46]:
def run(query:str):
    # 이번에는 리트리버로 context를 추출하는 부분을 체인의 내부가 아니라 체인 바깥에서 따로 실행함
    docs = retriever.get_relevant_documents(query)
    context = format_with_citations(docs)

    answer = chain.stream({"question": query, "context": context})
    for ans in answer:
        print(ans, end = '', flush=True)

    # print('\n\n')
    # print(context) # 디버깅용

run("MMORPG가 뭐야?")

<정의>: MMORPG는 수백 또는 수천 명 이상의 플레이어가 인터넷으로 같은 게임과 서버에 접속해 각자 역할을 맡아 플레이하는 대규모 다중 접속 온라인 역할 게임이다. 1990년대 후반 인터넷 보급과 함께 발전했으며, 플레이어가 게임을 일시 중단해도 지속되는 세계가 특징이다. 강제된 게임 목표 없이 이용자가 원하는 만큼 자유롭게 플레이할 수 있고, 협력 플레이에서 성취감과 소속감을 느낄 수 있다.  
<참고>: p.34-35

In [47]:
run('블랙웰')

<정의>: 블랙웰(Blackwell)은 엔비디아가 2024년에 공개한 신형 인공지능(AI) 반도체 아키텍처로, 호퍼(Hopper) 아키텍처의 후속 기술이다. 데이비드 헤롤드 블랙웰의 이름을 따서 명명되었으며, 2,080억 개의 트랜지스터가 집약되어 기존 H100 대비 2.5배 빠른 연산 속도를 자랑한다. B100 칩 두 개를 합친 B200을 연결해 단일 칩처럼 작동하며, 최신 NV링크 기술로 GPU 간 고속 통신을 지원한다. 구글, 메타, MS, 오라클, 테슬라, 오픈AI 등 글로벌 기업들이 블랙웰 플랫폼을 도입해 AI 및 자율주행 등 다양한 분야에 활용한다.

<참고>: p.70, p.71, p.15